In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:32836")
client

Client Scheduler: tcp://127.0.0.1:32836 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 64 Memory: 720.00 GB


In [43]:
import dask.dataframe as dd
from pathlib import Path
import glob

for csv_file in glob.glob("../../data/train/viznet/*.csv"):
    if "token" in csv_file:
        file_path = Path(csv_file)
        df = dd.read_csv(file_path)
        output_path = file_path.parent / file_path.stem
        output_path.mkdir(parents=True, exist_ok=True)
        df.to_json(str(output_path), compute=True, compression="gzip")

In [39]:
from elasticsearch import Elasticsearch, helpers

es = Elasticsearch()

In [40]:
es.indices.create(
        index="char_ngram",
        body={
            "settings": {"number_of_shards": 2},
            "mappings": {
                "properties": {
                    "text": {"type": "keyword"},
                    "count": {"type": "long"},
                }
            },
        },
        ignore=400,
    )

es.indices.create(
        index="tok_ngram",
        body={
            "settings": {"number_of_shards": 2},
            "mappings": {
                "properties": {
                    "text": {"type": "keyword"},
                    "count": {"type": "long"},
                }
            },
        },
        ignore=400,
    )

es.indices.create(
        index="reversed_indices",
        body={
            "settings": {"number_of_shards": 2},
            "mappings": {
                "properties": {
                    "text": {"type": "keyword"},
                    "indices": {"type": "nested"},
                }
            },
        },
        ignore=400,
    )

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'reversed_indices'}

In [ ]:
import rapidjson as json
import gzip

for sub_path in Path("../../data/train/viznet/ngram").iterdir():
    print(sub_path)
    if sub_path.is_dir() and "token" not in sub_path.name:
        for file_path in sub_path.iterdir():
            if file_path.suffix == ".part":
                with gzip.GzipFile(file_path, 'r') as fin:
                    data = [json.loads(l) for l in fin.readlines()]
                    helpers.bulk(es, data, index="char_gram")
    elif sub_path.is_dir():
        for file_path in sub_path.iterdir():
            if file_path.suffix == ".part":
                with gzip.GzipFile(file_path, 'r') as fin:
                    data = [json.loads(l) for l in fin.readlines()]
                    helpers.bulk(es, data, index="tok_ngram")

for sub_path in Path("../../data/train/viznet").iterdir():
    if sub_path.suffix == ".jsonl":
        with open(sub_path, 'r') as fin:
            data = [json.loads(l) for l in fin.readlines()]
            helpers.bulk(es, data, index="reversed_indices")